<a href="https://colab.research.google.com/github/tantien-hmtt/ml_sqli_api/blob/main/lstm_sqli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras.metrics import Recall

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df= pd.read_csv("/content/drive/MyDrive/data/sqliv2.csv", encoding='utf-16')
df.head()

,Sentence,Label
0,NaN,1
1,""" or pg_sleep ( __TIME__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' AND 1 = utl_inaddr.get_host_address ( ( S...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33761 entries, 0 to 33760
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  33757 non-null  object
 1   Label     33761 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 527.6+ KB


In [ ]:

df.dropna(inplace=True)#to drop out nan values

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33757 entries, 1 to 33760
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  33757 non-null  object
 1   Label     33757 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 791.2+ KB


In [ ]:
df.head()

,Sentence,Label
1,""" or pg_sleep ( __TIME__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' AND 1 = utl_inaddr.get_host_address ( ( S...,1
5,select * from users where id = '1' or @ @1 = ...,1


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 33757 entries, 1 to 33760
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  33757 non-null  object
 1   Label     33757 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 791.2+ KB


In [ ]:
X = df['Sentence']

In [ ]:
y = df['Label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

process data

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model


In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=[Recall()])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 150, 50)           50000     
                                                                 
 lstm_2 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_4 (Activation)   (None, 256)               0         
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [ ]:
model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
169/169 [==============================] - 55s 307ms/step - loss: 0.1377 - recall_2: 0.8607 - val_loss: 0.0320 - val_recall_2: 0.9800
Epoch 2/10
169/169 [==============================] - 50s 297ms/step - loss: 0.0280 - recall_2: 0.9870 - val_loss: 0.0347 - val_recall_2: 0.9811


In [ ]:
X_test_sequences = tok.texts_to_sequences(X_test)
X_test_sequences_matrix = sequence.pad_sequences(X_test_sequences,maxlen=max_len)

evaluate

In [ ]:
recall = model.evaluate(X_test_sequences_matrix,y_test)

211/211 [==============================] - 6s 31ms/step - loss: 0.0242 - recall_2: 0.9877


In [ ]:
print('Test set\n Loss: {:0.3f}\n Recall: {:0.3f}'.format(recall[0],recall[1]))

Test set
 Loss: 0.024
 Recall: 0.988


predict y

In [ ]:
y_pred=model.predict(X_test_sequences_matrix)

211/211 [==============================] - 5s 21ms/step


In [ ]:
print(y_pred)

[[1.4875193e-04]
 [6.2574363e-05]
 [1.1544379e-04]
 ...
 [8.3291595e-04]
 [1.7665551e-04]
 [1.3277333e-03]]


In [ ]:
for i in range(len(y_pred)):
    if y_pred[i]>0.5:
        y_pred[i]=1
    elif y_pred[i]<=0.5:
        y_pred[i]=0

In [ ]:
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


In [ ]:
confusion_matrix(y_test, y_pred)

array([[4467,    3],
       [  28, 2254]])

In [ ]:
accuracy=accuracy_score(y_test, y_pred)
precision=precision_score(y_test, y_pred)
recall=recall_score(y_test, y_pred)
print(f" Accuracy : {accuracy} \n Precision : {precision} \n Recall : {recall}")

 Accuracy : 0.9954087677725119 
 Precision : 0.9986708019494904 
 Recall : 0.9877300613496932


In [ ]:
from keras.models import load_model
from pathlib import Path


# Save neural network's trained weights
model.save_weights("/content/drive/MyDrive/model_lstm/v2_lstm_weights.h5")
#print('done')

# or you can save the full model via:
model.save('/content/drive/MyDrive/model_lstm/v2_lstm_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#load model and test

#load model

In [ ]:
import tensorflow as tf



model_path = '/content/drive/MyDrive/model_lstm/v2_lstm_model.h5'
weights_path = '/content/drive/MyDrive/model_lstm/v2_lstm_weights.h5'

try:
    loaded_model = tf.keras.models.load_model(model_path)
    loaded_model.load_weights(weights_path)
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading the model:", str(e))


Model loaded successfully.


#process data from user

In [ ]:
def clean_data(input_val):

    txts = tok.texts_to_sequences(input_val)
    input_val = sequence.pad_sequences(txts, maxlen=max_len)

    return input_val


def predict_sqli_attack():
    repeat=True

    space=''
    for i in range(20):
        space+= "-"

    print(space)
    input_val=input("Please enter some data: ")
    print(space)


    if input_val== '0':
        repeat=False

    sample_texts_list = []
    sample_texts_list.append(input_val)
    txts = tok.texts_to_sequences(sample_texts_list)
    txts = sequence.pad_sequences(txts, maxlen=max_len)
    print("txts: " + str(txts))
    result=loaded_model.predict(txts)

    print(result)
    print(space)

    if repeat == True:
            if result > 0.5:
                 print("ALERT! This can be SQL injection")
            elif result <= 0.5:
                 print("It seems to be a benign")

            print(space)

            predict_sqli_attack()

    elif repeat == False:
        print( " Closing ")

#implement

In [ ]:

predict_sqli_attack()


--------------------
--------------------
txts: [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0 741 121  18]]
1/1 [==============================] - 0s 26ms/step
[[0.00453213]]
--------------------
It seems to be a benign
--------------------
--------------------
--------------------
txts: [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   